In [1]:
%pip install -qU langchain langchain-core langchain-community langchain-openai langchain-anthropic langchainhub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.chat_models import init_chat_model

llm = init_chat_model('gpt-4o-mini', temperature=0)



c:\Users\samue\AppData\Local\pypoetry\Cache\virtualenvs\sociaty-newsletter-generator-brUSjMkx-py3.12\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [5]:
from sociaty_newsletter_generator.models import (
    Cluster,
    ClusteringSession,
    SetOfUniqueArticles,
    Article,
)

# Database utilities


In [6]:
from sociaty_newsletter_generator.db import init_db

await init_db()

In [7]:
sessions = await ClusteringSession.find_many().to_list()

for session in sessions:
    print(session.id)
    
    start, end = session.data_start, session.data_end
    nb_days = (end - start).days
    print(f" - {start.strftime('%Y-%m-%d')} -> {end.strftime('%Y-%m-%d')} ({nb_days} days)")
    print(f" - {session.clusters_count} total clusters")

    included_clusters = await session.get_included_sorted_clusters()
    print(f" - {len(included_clusters)} included clusters")
    
    print(" - Example of good clusters")
    for cluster in included_clusters[:10]:
        print(f"    - {cluster.title}")

    print()



6697a91d6720e5b25ef7946b
 - 2024-06-29 -> 2024-07-02 (3 days)
 - 109 total clusters
 - 73 included clusters
 - Example of good clusters
    - SK Hynix's $75 Billion Investment Focuses on AI-Enhanced HBM Processors
    - Controversy Surrounding AI Training on Web Content Sparks Legal Battles
    - Morgan Freeman Condemns Unauthorized AI Voice Imitations
    - Apple Working to Integrate AI Features into Vision Pro Headsets
    - States Addressing AI Skills Gap in Workforce
    - Apple Plans to Monetize Advanced AI Features with Subscription Model
    - Apple to Integrate Google's Gemini AI into iPhone for Enhanced AI Capabilities
    - Chinese AI Startups Flock to Singapore for Global Expansion Amid US Restrictions
    - Top Artificial Intelligence (AI) Stocks for Long-Term Investment
    - Robinhood Enhances Investing App with AI Tools Through Acquisition of Pluto Capital

6697ab4a6720e5b25ef794d9
 - 2024-07-02 -> 2024-07-05 (3 days)
 - 264 total clusters
 - 152 included clusters
 - Exa

# Summarize one subject


In [8]:
from langchain import hub
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.language_models import BaseChatModel
from langsmith import traceable


class ChainOfDensity:
    """
    A class that wraps the Chain of Density (CoD) prompting technique to iteratively forces the model to pack more detail into a fixed length summary.
    """

    def __init__(
        self,
        model: BaseChatModel,
        content_category: str = "Article",
        entity_range: str = "2-3",
        max_words: int = 100,
        iterations: int = 6,
    ):
        self.model = model
        self.content_category = content_category
        self.entity_range = entity_range
        self.max_words = max_words
        self.iterations = iterations
        self._chain = self._get_chain()

    def _get_chain(self):
        prompt = hub.pull("supermuel/chain-of-density-prompt-multilingual")
        json_parser = SimpleJsonOutputParser()
        return prompt | self.model | json_parser | self._extract_last_summary

    def _get_input(self, content) -> dict:
        return {
            "content": content,
            "content_category": self.content_category,
            "entity_range": self.entity_range,
            "max_words": self.max_words,
            "iterations": self.iterations,        }

    @staticmethod
    def _extract_last_summary(output: list[dict]):
        try:
            return output[-1]["denser_summary"]
        except IndexError:
            raise Exception("Invalid format returned from the model : No output.")
        except KeyError:
            raise Exception(
                "Invalid format returned from the model : No denser_summary key."
            )
        except Exception as e:
            raise Exception(f"Invalid format returned from the model : {e}")

    @traceable
    def generate_summary(self, content: str) -> str:
        """
        Generate a dense summary of the given content.
        """

        return self._chain.invoke(self._get_input(content))


In [9]:
session_id ="669a345da02feb1858343077"
session = await ClusteringSession.get(session_id)
assert session

clusters = await session.get_included_sorted_clusters()

first_cluster = clusters[0]
print(f"{first_cluster.title} ({first_cluster.articles_count} articles)")
print(first_cluster.summary)

articles = SetOfUniqueArticles(await first_cluster.get_articles())

def format_articles(articles: SetOfUniqueArticles):
    return "\n\n".join(
        f"{article.title}\n{article.date}\n{article.url}\n{article.body}"
       for article in articles
    )

print(format_articles(articles))

OpenAI Introduces GPT-4o Mini: A Cheaper and More Accessible AI Model (71 articles)
OpenAI has unveiled GPT-4o mini, a smaller and more affordable AI model that offers advanced intelligence capabilities at a lower cost. This new model is 60% cheaper than previous versions, making it accessible to a wider range of developers and users.
OpenAI unveils GPT-4o mini, a smaller and cheaper AI model
2024-07-18 15:34:00
https://techcrunch.com/2024/07/18/openai-unveils-gpt-4o-mini-a-small-ai-model-powering-chatgpt/
OpenAI introduced GPT-4o mini on Thursday, its latest small AI model. The company says GPT-4o mini, which is cheaper and faster than OpenAI's current

ChatGPT Maker OpenAI Goes Smaller and Cheaper with New AI Tech
2024-07-18 16:58:00
https://www.wsj.com/tech/ai/chatgpt-maker-goes-smaller-and-cheaper-with-new-ai-tech-0cbcff84?mod=tech_lead_pos1
OpenAI released a smaller and cheaper version of the technology that powers ChatGPT, part of an industry shift toward more efficient artificia

In [10]:
from langchain_core.prompts import PromptTemplate
from textwrap import dedent

from langchain_core.pydantic_v1 import BaseModel, Field

class SummaryOutput(BaseModel):
    """Output for the summarization process. Every field is mandatory."""
    scratchpad: str = Field(..., description="Very dense set of facts and ideas separated by comas that should cover a large amount of the content.")
    missing_entities: str = Field(..., description="Mandatory list of ideas or facts that you forgot to include in the scratchpad.")
    final_summary: str = Field(..., description="Final very dense summary of the content including the scratchpad and forgot facts.")

@traceable
def simple_summary(articles : SetOfUniqueArticles,llm:BaseChatModel)->SummaryOutput:
    prompt = PromptTemplate.from_template(
        dedent("""Summarize this set of news articles : 
        <articles>
        {formatted_articles}
        </articles>
    """
        )
    )

    structured_llm = llm.with_structured_output(SummaryOutput)

    formatted_articles = format_articles(articles)

    chain = prompt|structured_llm

    return chain.invoke({'formatted_articles': formatted_articles})

summary = simple_summary(articles=articles,llm=llm)
print(summary)

scratchpad="OpenAI has launched GPT-4o mini, a smaller and cheaper AI model designed to be more accessible and efficient for developers. This new model is 60% cheaper than GPT-3.5 Turbo, priced at 15 cents per million input tokens and 60 cents per million output tokens. It outperforms competitors like Google's Gemini 1.5 Flash and Anthropic's Claude 3 Haiku in various benchmarks, scoring 82% on the MMLU benchmark. The model supports text and vision inputs, with plans for future multimodal capabilities. GPT-4o mini aims to democratize AI technology, making it easier for smaller developers to integrate advanced AI into their applications. It is now available for free users and subscribers of ChatGPT, replacing the older GPT-3.5 model. OpenAI's move reflects a broader industry trend towards more cost-effective AI solutions." missing_entities='GPT-4o mini has a context window of 128,000 tokens, supports up to 16K output tokens per request, and has a knowledge cutoff of October 2023. It is 

In [34]:
# for cluster in clusters[:10]:
#     print(f"{cluster.title}")
#     summary = simple_summary(articles=await cluster.get_articles(),llm=llm)
#     print(summary)
#     print("\n"*3)

OpenAI Introduces GPT-4o Mini: A Cheaper and More Accessible AI Model
scratchpad="OpenAI has launched GPT-4o mini, a smaller and cheaper AI model designed to make advanced AI technology more accessible. This new model is 60% cheaper than GPT-3.5 Turbo and outperforms competitors like Google's Gemini 1.5 Flash and Anthropic's Claude 3 Haiku on various benchmarks. GPT-4o mini supports text and vision inputs, with plans to include audio and video capabilities in the future. It is aimed at developers with limited budgets, allowing them to leverage generative AI for their applications. The model is priced at 15 cents per million input tokens and 60 cents per million output tokens, making it a cost-effective option for businesses. OpenAI's move reflects a broader industry trend towards more efficient AI models, catering to a wider range of customers and enhancing competition in the AI market." missing_entities='GPT-4o mini has a context window of 128,000 tokens, supports up to 16K output tok

### Chain of density summary (Won't use that)


In [12]:
def chain_of_density_summary(articles:SetOfUniqueArticles,llm:BaseChatModel)->str:
    cod = ChainOfDensity(llm, 'set of news articles', 
                         entity_range='5-6',
                         iterations=4,
                         max_words=600,
                         )

    return cod.generate_summary(content=format_articles(articles))

# for cluster in clusters[:3]:
#     print(f"{cluster.title}")
#     summary = chain_of_density_summary(articles=await cluster.get_articles(),llm=llm)
#     print(summary)
#     print("\n"*3)

In [ ]:
### 

# Langchain


plan :
we get a list of
